# Исследование данных об инвестиции венчурных фондов в компании-стартапы

**Данные**

В проекте работа идёт с базой данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы. Эта база данных основана на датасете Startup Investments, опубликованном на популярной платформе для соревнований по исследованию данных Kaggle.

**Задача**

Проанализировать данные о фондах и инвестициях, произвести выгрузки данных и ответить на поставленные вопросы с помощью SQL.

# Задания

## Задание 1

Посчитайте, сколько компаний закрылось.

```
SELECT COUNT(status) AS closed_company
FROM company
WHERE status = 'closed'
```



## Задание 2

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.



```
SELECT funding_total
FROM company
WHERE country_code = 'USA'
      AND category_code = 'news'
ORDER BY funding_total DESC
```



## Задание 3


Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.



```
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
      AND EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2011 AND 2013
```



## Задание 4

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на `Silver`.



```
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'
```



## Задание 5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `money`, а фамилия начинается на `K`.



```
SELECT *
FROM people
WHERE last_name LIKE 'K%'
      AND twitter_username LIKE '%money%'
```



## Задание 6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.



```
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC
```



## Задание 7

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.



```
SELECT funded_at,
       MIN(raised_amount) ,
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0
       AND MIN(raised_amount) <> MAX(raised_amount)
```



## Задание 8

Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.
Отобразите все поля таблицы `fund` и новое поле с категориями.



```
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END
FROM fund
```



## Задание 9

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.



```
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS avg_rounds
       
FROM fund
GROUP BY activity
ORDER BY avg_rounds;
```



## Задание 10

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.



```
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10
```



## Задание 11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна



```
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id
```



## Задание 12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.



```
SELECT c.name,
       COUNT(DISTINCT(e.instituition))
FROM company AS c
LEFT JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON e.person_id=p.id
GROUP BY c.name
ORDER BY COUNT(DISTINCT(e.instituition)) DESC
LIMIT 5
  
```



## Задание 13

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.



```
SELECT DISTINCT(c.name)
FROM company AS c
LEFT JOIN funding_round as fr ON c.id=fr.company_id
WHERE fr.is_first_round = 1
      AND fr.is_last_round = 1
      AND c.status = 'closed'
```



## Задание 14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.



```
SELECT DISTINCT(p.id)
FROM people as p
JOIN (SELECT DISTINCT(c.id)
        FROM company AS c
        LEFT JOIN funding_round as fr ON c.id=fr.company_id
        WHERE fr.is_first_round = 1
                 AND fr.is_last_round = 1
                 AND c.status = 'closed') as z ON p.company_id = z.id
```



## Задание 15

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.



```
SELECT DISTINCT(p.id),
        e.instituition
FROM people as p
JOIN (SELECT DISTINCT(c.id)
             FROM company AS c
             LEFT JOIN funding_round as fr ON c.id=fr.company_id
             WHERE fr.is_first_round = 1
                     AND fr.is_last_round = 1
                     AND c.status = 'closed') as z ON p.company_id = z.id
JOIN education AS e ON e.person_id=p.id
```



## Задание 16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.



```
SELECT DISTINCT(p.id),
       COUNT(instituition)
FROM people as p
JOIN (SELECT DISTINCT(c.id)
             FROM company AS c
             LEFT JOIN funding_round as fr ON c.id=fr.company_id
             WHERE fr.is_first_round = 1
                     AND fr.is_last_round = 1
                     AND c.status = 'closed') as z ON p.company_id = z.id
JOIN education AS e ON e.person_id=p.id
GROUP BY p.id
```



## Задание 17

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.



```
SELECT AVG(x.inst)
FROM (SELECT DISTINCT(p.id),
       COUNT(instituition) AS inst
FROM people as p
JOIN (SELECT DISTINCT(c.id)
             FROM company AS c
             LEFT JOIN funding_round as fr ON c.id=fr.company_id
             WHERE fr.is_first_round = 1
                     AND fr.is_last_round = 1
                     AND c.status = 'closed') as z ON p.company_id = z.id
JOIN education AS e ON e.person_id=p.id
GROUP BY p.id) AS x
```



## Задание 18

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)



```
SELECT AVG(x.inst)
FROM (SELECT DISTINCT(p.id),
       COUNT(instituition) AS inst
FROM people as p
JOIN (SELECT DISTINCT(c.id)
             FROM company AS c
             WHERE c.name='Facebook') as z ON p.company_id = z.id
JOIN education AS e ON e.person_id=p.id
GROUP BY p.id) AS x
```



## Задание 19

Составьте таблицу из полей:
* `name_of_fund` — название фонда;
* `name_of_company` — название компании;
* `amount` — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.



```
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
JOIN company AS c ON i.company_id=c.id
JOIN fund AS f ON f.id=i.fund_id
JOIN funding_round AS fr ON fr.id=i.funding_round_id

WHERE c.milestones > 6
      AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN 2012 AND 2013
```



## Задание 20

Выгрузите таблицу, в которой будут такие поля:
* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.



```
SELECT c.name,
       a.price_amount,
       c1.name AS name_1,
       c1.funding_total,
       ROUND(a.price_amount/c1.funding_total)
FROM acquisition AS a
LEFT JOIN company AS c ON c.id=a.acquiring_company_id
LEFT JOIN company AS c1 ON c1.id=a.acquired_company_id
WHERE a.price_amount>0
      AND c1.funding_total>0
ORDER BY a.price_amount DESC, c1.name
LIMIT 10
```



## Задание 21

Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.



```
SELECT c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp))
FROM company AS c
JOIN funding_round AS fr ON fr.company_id = c.id
WHERE category_code = 'social'
      AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN 2010 AND 2013
      AND fr.raised_amount > 0
```





## Задание 22

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.



```
WITH count_fund AS (
SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)) AS month_round,
       COUNT(DISTINCT f.name) AS count_found
FROM investment as i
JOIN funding_round AS fr ON fr.id=i.funding_round_id
JOIN fund AS f ON i.fund_id=f.id
WHERE fund_id IN (SELECT id
                  FROM fund
                  WHERE country_code = 'USA')
        AND i.funding_round_id IN (SELECT id 
                                   FROM funding_round
                                   WHERE EXTRACT(YEAR FROM CAST(funded_at AS timestamp)) BETWEEN 2010 AND 2013)
        GROUP BY month_round),

count_company AS (
SELECT EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS month_acquired,
    COUNT(acquired_company_id) AS count_company,
    SUM(price_amount) AS sum_acquired
FROM acquisition
WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2010 AND 2013
GROUP BY month_acquired)

SELECT cf.month_round AS month,
       cf.count_found,
       cc.count_company,
       cc.sum_acquired
FROM count_fund AS cf 
    FULL JOIN count_company AS cc ON cc.month_acquired=cf.month_round
    
```



## Задание 23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.



```
WITH
invest_2011 AS (
SELECT country_code,
       AVG(funding_total) AS investment_2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = '2011'
GROUP BY country_code
),
invest_2012 AS (
SELECT country_code,
       AVG(funding_total) AS investment_2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = '2012'
GROUP BY country_code
),
invest_2013 AS (
SELECT country_code,
       AVG(funding_total) AS investment_2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = '2013'
GROUP BY country_code
)
SELECT i2011.country_code,
       i2011.investment_2011,
       i2012.investment_2012,
       i2013.investment_2013
FROM invest_2011 AS i2011
JOIN invest_2012 AS i2012 ON i2011.country_code=i2012.country_code
JOIN invest_2013 AS i2013 ON i2011.country_code=i2013.country_code
ORDER BY i2011.investment_2011 DESC

```

